## cik==1422183 scratch work


In [72]:
import re
import pandas as pd
import numpy as np
import platform
from utils import arguements,init_logger,ROOT_PATH
from fuzzywuzzy import process

def get_standard_name(col, choices, score_cutoff=60):
    best_match, score = process.extractOne(col, choices)
    if score > score_cutoff:
        return best_match
    return col
            
def merge_duplicate_columns(
    df:pd.DataFrame,
)->pd.DataFrame:
    duplicate_cols = df.columns[df.columns.duplicated(keep=False)]
    for col_name in duplicate_cols.unique():
        duplicate_data = df.loc[:, df.columns == col_name]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, df.columns != col_name]
        df[col_name] = merged_data
    return df

def standard_field_names()->tuple:
    return (
        'portfolio',
        'footnotes',
        'industry',
        'rate',
        'floor',
        'maturity',
        'principal amount', # TODO change stand names for more dynamic fuzzywuzzy matching
        'cost',
        'value',
        'investment',
        'date',
        'subheaders',
        'number of shares'
    )

def common_subheaders()->tuple:
    return (
        'senior secured loans',
        'first lien',
        'second lien',
        'senior secured bonds',
        'subordinated debt',
        'equity/other',
        'collateralized securities',
        'preferred equity—',
        'control investments',
        'affiliated investments',
        'non-control/non-affilate investments'
    )
    
def strip_string(
    columns_names:list,
    standardize:bool=False
)->tuple:
    columns = tuple(map(lambda col:re.sub(r'[^a-z]', '', str(col).lower()),columns_names))
    if standardize:
        standard_fields = standard_field_names()
        return tuple(
            get_standard_name(col,standard_fields) for col in columns
        )
    return columns

def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

def get_key_fields(
    df_cur:pd.DataFrame
)->tuple:
    important_fields = standard_field_names() + common_subheaders()
    for idx,row in enumerate(df_cur.iterrows()):
        found = any(any(
            key in str(field).lower() 
            for key in important_fields)
                    for field in row[-1].dropna().tolist()
            )
        if found and len(set(row[-1].dropna().tolist())) >= 5:
            fields = strip_string(row[-1].tolist(),standardize=found) ,idx
            return fields
    return strip_string(df_cur.iloc[0].tolist(),standardize=found),0



def extract_subheaders(
    df:pd.DataFrame,
)->pd.DataFrame:
    include = df.apply(
    lambda row: row.astype(str).str.contains('|'.join(common_subheaders()), case=False, na=False).any(),
        axis=1) # 
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total', case=False, na=False).any(),
        axis=1
    )
    
    idx = df[include & exclude].index.tolist()
    df['subheaders'] = 'no_subheader'
    
    if not idx:
        return df
    
    df.loc[idx[-1]:,'subheaders'] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],'subheaders'] = subheader
    df.drop(idx,axis=0,inplace=True,errors='ignore') # drop subheader row
    return df

#TODO is number of shares = principal amount
def clean(
    file:str,
)->list:
    dirs = file.split('/') if platform.system() == "Linux" else file.split('\\')
    if  len(dirs) < 3 or '.csv' not in dirs[-1]:
        return
    df_cur = pd.read_csv(file,encoding='utf-8')
    df_cur = df_cur.T.drop_duplicates().T
    if df_cur.shape[1] < 4:
        return
    if df_cur.empty:
        return
    
    df_cur.reset_index(drop=True,inplace=True)
    
    important_fields,idx = get_key_fields(df_cur)
    if len(set(important_fields) - {''}) < 4:
        df_cur.replace('\u200b', np.nan, regex=True,inplace=True)
        df_cur.replace(r'\$|€|£',np.nan,regex=True,inplace=True)
        columns_to_drop = df_cur.notna().sum() == 1 #or df_cur.applymap(lambda x: ('$' == x)).any()
        return df_cur.iloc[:,1:].drop(columns=columns_to_drop[columns_to_drop].index)
    
    df_cur.columns = important_fields
    df_cur = merge_duplicate_columns(df_cur)
    cols_to_drop = [
        col for col in df_cur.columns.tolist() 
        if col == '' or col == 'nan'
    ] 

    df_cur.drop(columns=cols_to_drop, errors='ignore',inplace=True) # drop irrelevant columns
    return df_cur

In [321]:
dfs = [df1,df2,df3]
final_columns = max(dfs,key=lambda col:len(col.columns)).columns
final_columns

Index(['Unnamed: 0', '0', '1', '2', '4', '7', '10'], dtype='object')

In [318]:
df1 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-12-31\output\cleaned_0.csv")
df2 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-12-31\output\cleaned_1.csv")
df3 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-12-31\output\cleaned_2.csv")
date_final = pd.DataFrame(concat(df1, df2, df3.drop(columns=['Unnamed: 0'])))
date_final.columns = df1.columns
date_final.tail(50)


,Unnamed: 0,portfolio,industry,principal amount,cost,value
56,56,"Chaparral Energy Inc., 8.3%, 9/1/21 (d)",Upstream,4000,4087,4340
57,57,"Chaparral Energy Inc., 7.6%, 11/15/22 (d)",Upstream,9000,9299,9450
58,58,"Comstock Resources, Inc., 9.5%, 6/15/20 (d)(f)",Upstream,12000,11463,12744
59,59,"Crestwood Midstream Partners L.P., 7.8%, 4/1/...",Midstream,10000,10098,10413
60,60,"CrownRock, L.P., 10.0%, 8/15/16",Upstream,10000,10689,10738
61,0,NaN,NaN,NaN,NaN,NaN
62,1,Portfolio Company (a),Industry,Principal Amount (b),Amortized Cost,Fair Value (c)
63,2,"EP Energy LLC, 7.8%, 9/1/22 (d)",Upstream,"$6,600",$ 6624,7054 $
64,3,"EPE Holdings LLC, 8.1%, 12/15/17 (d)",Upstream,6000,5970,5948
65,4,"EPL Oil & Gas, Inc., 8.3%, 2/15/18 (d)(f)",Upstream,3200,3169,3300


In [327]:
temp2.columns[1] + temp2.columns[2]

'industry'

In [324]:
temp2 = clean(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-12-31\Schedule_of_Investments_2.csv")
temp2.columns = ['portfolio','','industry','principal amount','cost','value']
temp2

,portfolio,,industry,principal amount,cost,value
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Number of Shares,Amortized Cost,Fair Value (c)
2,Equity/Other—7.6% (g),NaN,NaN,NaN,NaN,NaN
3,"Fortune Creek Co-Invest I L.P., LP Interest (...",NaN,Midstream,NaN,23345,22619
4,"Plains Offshore Operations Inc., Preferred Equ...",NaN,Upstream,209227,20776,22370
5,NaN,NaN,Upstream,405378,689,973
6,NaN,NaN,NaN,NaN,NaN,NaN
7,Total Equity/Other,NaN,NaN,NaN,44810,45962
8,NaN,NaN,NaN,NaN,NaN,NaN
9,TOTAL INVESTMENTS—116.3%,NaN,NaN,NaN,677919,701172


In [310]:
temp1 = clean(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\Schedule_of_Investments_3.csv")
temp1

,Unnamed: 0,0,1,3,6,9,15,20,26,32
0,0,NaN,NaN,NaN,NaN,NaN,NaN,Number of Shares,Amortized Cost,Fair Value (d)
1,1,Preferred Equity—27.0% (l),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Abaco Energy Technologies LLC, Preferred Equity",NaN,(o)(w)(x),Service & Equipment,NaN,NaN,28942003,1447,14312
3,3,"Altus Midstream LP, Series A Preferred Units",NaN,(j)(w)(x),Midstream,11.0%,6/28/26,50000,50603,51000
4,4,"Altus Power America Holdings, LLC, Preferred E...",NaN,(p)(w)(x),Power,"9.0%, 5.0% PIK (5.0% Max PIK)",10/3/23,29908537,29909,29909
5,5,"Global Jet Capital Holdings, LP, Preferred Equity",NaN,(o)(w)(x),Industrials,NaN,NaN,27856,2786,209
6,6,"Great Western Petroleum, LLC, Preferred Equity",NaN,(n)(w)(x),Upstream,15.5%,12/31/27,36364,41468,37724
7,7,"Limetree Bay Ventures, LLC, Preferred Equity",NaN,(w)(x),Midstream,13.9%,11/30/24,75000000,74373,69705
8,8,"MB Precision Investment Holdings LLC, Class A ...",NaN,(o)(w)(x)(z),Industrials,NaN,NaN,8952623,1880,1101
9,9,"NGL Energy Partners, LP, Preferred Equity",NaN,(k)(w)(x),Midstream,14.2%,7/2/27,125000,120734,120113


In [311]:
temp = clean(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\Schedule_of_Investments_4.csv")
temp

,Unnamed: 0,0,1,2,4,13,16,19
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,Number of Shares,Amortized Cost,Fair Value (c)
2,2,Equity/Other—7.4% (i),NaN,NaN,NaN,NaN,NaN,NaN
3,3,"American Energy Ohio Holdings, LLC, Common Equity",NaN,(j)(k),Upstream,7788845,7789,7789
4,4,"BBH Operating LLC, Common Equity",NaN,(l)(k),Upstream,1000,1000,995
5,5,"Fortune Creek Co-Invest I L.P., LP Interest",NaN,(h)(m),Midstream,NaN,22773,23417
6,6,"Plains Offshore Operations Inc., Preferred Equity",NaN,(d),Upstream,20000,21798,24602
7,7,"Plains Offshore Operations Inc., Warrants",NaN,(d)(k),Upstream,405378,689,1176


In [299]:
df1 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\output\cleaned_0.csv")
df2 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\output\cleaned_1.csv")
df3 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\output\cleaned_2.csv")
df4 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\output\cleaned_3.csv")
df5 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\output\cleaned_5.csv")
temp = clean(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2013-09-30\Schedule_of_Investments_4.csv")

final_columns = df1.columns
date_final = pd.DataFrame(concat(df1, df2, df3, df4,temp,df5))
date_final.columns = final_columns
date_final.drop(date_final.columns[0],axis=1,inplace=True)
date_final = extract_subheaders(date_final)
date_final.tail(50)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pysol\\Desktop\\projects\\sec_filings\\1501729\\2013-09-30\\output\\cleaned_0.csv'

In [102]:
df1 = clean(r'C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-09-30\Schedule_of_Investments_0.csv')
df2 = clean(r'C:\Users\pysol\Desktop\projects\sec_filings\1501729\2012-09-30\Schedule_of_Investments_1.csv')
df1

C:\Users\pysol\AppData\Local\Temp\ipykernel_12584\2000200988.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data
C:\Users\pysol\AppData\Local\Temp\ipykernel_12584\2000200988.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data


,portfolio,industry,principal amount,cost,value
0,NaN,NaN,,,
1,Portfolio Company (a),Industry,Principal Amount (b),Amortized Cost,Fair Value (c)
2,Senior Secured Loans—First Lien—36.3%,NaN,,,
3,"Crestwood Holdings LLC, L+825, 1.5% LIBOR Floo...",Midstream,$ 21634,$ 21773,$ 22002
4,"Dynegy Midwest Generation LLC (CoalCo), L+775,...",Power,5676,5752,5963
...,...,...,...,...,...
62,"Samson Investment Co., 9.8%, 2/15/20 (d)",Upstream,25000,25034,25834
63,"SandRidge Energy, Inc., 7.5%, 3/15/21 (d)(f)",Upstream,1250,1305,1291
64,NaN,NaN,,,
65,Total Subordinated Debt,NaN,,217424,223638


In [103]:
df2

,portfolio,industry,number of shares,cost,value
0,NaN,NaN,,,
1,Portfolio Company (a),Industry,Number of Shares,Amortized Cost,Fair Value (c)
2,Equity/Other—10.9% (g),NaN,,,
3,"Fortune Creek Co-Invest I L.P., LP Interest (...",Midstream,,$ 22967,$ 23239
4,"Plains Offshore Operations Inc., Preferred Equ...",Upstream,208137,20454,20947
5,"Plains Offshore Operations Inc., Strike: $20.0...",Upstream,405378,689,872
6,NaN,NaN,,,
7,Total Equity/Other,NaN,,44110,45058
8,NaN,NaN,,,
9,TOTAL INVESTMENTS—110.8%,NaN,,444144 $,455845


In [104]:
df1.columns,df2.columns

(Index(['portfolio', 'industry', 'principal amount', 'cost', 'value'], dtype='object'),
 Index(['portfolio', 'industry', 'number of shares', 'cost', 'value'], dtype='object'))

In [91]:
# df = pd.DataFrame(df1.values.tolist()+df2.values.tolist())
# df.columns = df1.columns
# df = pd.concat([df1,df2],axis=0)
df = pd.DataFrame(concat(df1,df2))
df

,0,1,2,3,4
0,NaN,NaN,,,
1,Portfolio Company (a),Industry,Principal Amount (b),Amortized Cost,Fair Value (c)
2,Senior Secured Loans—First Lien—36.3%,NaN,,,
3,"Crestwood Holdings LLC, L+825, 1.5% LIBOR Floo...",Midstream,$ 21634,$ 21773,$ 22002
4,"Dynegy Midwest Generation LLC (CoalCo), L+775,...",Power,5676,5752,5963
...,...,...,...,...,...
77,NaN,NaN,,,
78,LIABILITIES IN EXCESS OF OTHER ASSETS—(10.8%),NaN,,,"(44,522"
79,NaN,NaN,,,
80,NET ASSETS—100.0%,NaN,,,411323 $


In [236]:
temp1.head()

,Unnamed: 0,0,1,3,6,9,15,20,26,32
0,0,NaN,NaN,NaN,NaN,NaN,NaN,Number of Shares,Amortized Cost,Fair Value (d)
1,1,Preferred Equity—27.0% (l),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Abaco Energy Technologies LLC, Preferred Equity",NaN,(o)(w)(x),Service & Equipment,NaN,NaN,28942003,1447,14312
3,3,"Altus Midstream LP, Series A Preferred Units",NaN,(j)(w)(x),Midstream,11.0%,6/28/26,50000,50603,51000
4,4,"Altus Power America Holdings, LLC, Preferred E...",NaN,(p)(w)(x),Power,"9.0%, 5.0% PIK (5.0% Max PIK)",10/3/23,29908537,29909,29909


In [241]:
df1 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\output\cleaned_0.csv")
df2 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\output\cleaned_1.csv")
df3 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\output\cleaned_2.csv")
temp1 = clean(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\Schedule_of_Investments_3.csv")
df5 = pd.read_csv(r"C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\output\cleaned_4.csv")
final_columns = df1.columns
date_final = pd.DataFrame(concat(df1, df2, df3, temp1,df5))
date_final.columns = final_columns
date_final.drop(date_final.columns[0],axis=1,inplace=True)
date_final = extract_subheaders(date_final)
date_final.tail(50)

,portfolio,footnotes,industry,rate,maturity,floor,principal amount,cost,value,subheaders
113,"Limetree Bay Ventures, LLC, Preferred Equity",NaN,(w)(x),Midstream,13.9%,11/30/24,75000000,74373,69705,Preferred Equity—27.0% (l)
114,"MB Precision Investment Holdings LLC, Class A ...",NaN,(o)(w)(x)(z),Industrials,NaN,NaN,8952623,1880,1101,Preferred Equity—27.0% (l)
115,"NGL Energy Partners, LP, Preferred Equity",NaN,(k)(w)(x),Midstream,14.2%,7/2/27,125000,120734,120113,Preferred Equity—27.0% (l)
116,"NuStar, Preferred Equity",NaN,(k)(w)(x),Midstream,12.8%,6/29/28,5910165,150054,184929,Preferred Equity—27.0% (l)
117,"Rosehill Resources, Inc. Preferred Equity",NaN,(o)(w)(x),Upstream,NaN,NaN,2536,2511,2803,Preferred Equity—27.0% (l)
118,"Segreto Power Holdings, LLC, Preferred Equity",NaN,(g)(w)(x),Power,13.1%,5/8/25,70297,69331,81313,Preferred Equity—27.0% (l)
119,"Synergy Offshore LLC, Preferred Equity",NaN,(m)(o)(v)(w)(x),Upstream,NaN,NaN,71131,93009,5000,Preferred Equity—27.0% (l)
120,"TE Holdings, LLC, Preferred Equity",NaN,(o)(x),Upstream,NaN,NaN,1475531,14734,—,Preferred Equity—27.0% (l)
121,"USA Compression Partners, LP, Preferred Equity",NaN,(k)(w)(x),Midstream,9.8%,4/3/28,79336,77466,84728,Preferred Equity—27.0% (l)
122,Total Preferred Equity,NaN,NaN,NaN,NaN,NaN,NaN,730305,682846,Preferred Equity—27.0% (l)


In [207]:
date_final.shape

(158, 10)

In [137]:
df_cur = clean(r'C:\Users\pysol\Desktop\projects\sec_filings\1501729\2019-09-30\Schedule_of_Investments_3.csv')
df_cur

,0,3,6,9,15,20,26,32
0,<NA>,<NA>,<NA>,<NA>,<NA>,Number of Shares,Amortized Cost,Fair Value (d)
1,Preferred Equity—27.0% (l),<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,"Abaco Energy Technologies LLC, Preferred Equity",(o)(w)(x),Service & Equipment,<NA>,<NA>,28942003,1447,14312
3,"Altus Midstream LP, Series A Preferred Units",(j)(w)(x),Midstream,11.0%,6/28/26,50000,50603,51000
4,"Altus Power America Holdings, LLC, Preferred E...",(p)(w)(x),Power,"9.0%, 5.0% PIK (5.0% Max PIK)",10/3/23,29908537,29909,29909
5,"Global Jet Capital Holdings, LP, Preferred Equity",(o)(w)(x),Industrials,<NA>,<NA>,27856,2786,209
6,"Great Western Petroleum, LLC, Preferred Equity",(n)(w)(x),Upstream,15.5%,12/31/27,36364,41468,37724
7,"Limetree Bay Ventures, LLC, Preferred Equity",(w)(x),Midstream,13.9%,11/30/24,75000000,74373,69705
8,"MB Precision Investment Holdings LLC, Class A ...",(o)(w)(x)(z),Industrials,<NA>,<NA>,8952623,1880,1101
9,"NGL Energy Partners, LP, Preferred Equity",(k)(w)(x),Midstream,14.2%,7/2/27,125000,120734,120113


In [27]:
index_list = df_cur.apply(
    lambda row:row.astype(str).str.contains('total investments', case=False, na=False).any(),
    axis=1
)
index_list.sum()

1

In [28]:
include = df_cur.apply(
    lambda row: row.astype(str).str.contains('|'.join(common_subheaders()), case=False, na=False).any(),
    axis=1) # 
exclude = ~df_cur.apply(
    lambda row: row.astype(str).str.contains('total', case=False, na=False).any(),
    axis=1
)



In [29]:
df_cur[include & exclude]

,industry,portfolio,principal amount,cost,value
2,NaN,Senior Secured Loans—First Lien—44.2%,,,
13,NaN,Senior Secured Loans—Second Lien—17.2%,,,
20,NaN,Senior Secured Bonds—14.9%,,,
27,NaN,Subordinated Debt—31.2%,,,


In [258]:
df = pd.read_csv(r'C:\Users\pysol\Desktop\projects\sec_filings\1396440\2023-09-30\Schedule_of_Investments_0.csv')
df.replace(['\u200b','',')',0],np.nan,inplace=True)
df.dropna(axis=1,how='all',inplace=True)
df.dropna(axis=0,how='all',inplace=True)
df.columns = df.iloc[0].str.replace('[^a-zA-Z]', '', regex=True)
df = merge_duplicate_columns(df)
df.replace(['\u200b','',')',0],np.nan,inplace=True)
df.drop(columns=df.columns[pd.isna(df.columns)].tolist() + [col for col in df.columns if col == ''],axis=1,inplace=True)
for i in range(3):
    df[df.columns[i]].fillna(method='ffill',inplace=True)
    print(df[df.columns[i]].unique())
df

['Portfolio Company (1) (20)' 'Control Investments (5)'
 'Analytical Systems Keco Holdings, LLC' 'ASC Interests, LLC'
 'ATS Workholding, LLC' 'Barfly Ventures, LLC' 'Batjer TopCo, LLC'
 'Bolder Panther Group, LLC' 'Brewer Crane Holdings, LLC']
['Business Description' 'Manufacturer of Liquid and Gas Analyzers'
 'Recreational and Educational Shooting Facility'
 'Manufacturer of Machine Cutting Tools and Accessories'
 'Casual Restaurant Group' 'HVAC Mechanical Contractor'
 'Consumer Goods and Fuel Retailer'
 'Provider of Crane Rental and Operating Services']
['Type of Investment (2)\xa0(3)\xa0(15)' 'Secured Debt'
 'Preferred Member Units' 'Warrants' 'Member Units' 'Preferred Stock'
 'Class B Preferred Member Units']


1,PortfolioCompany,BusinessDescription,TypeofInvestment,InvestmentDate,SharesUnits,TotalRate,ReferenceRateandSpread,PIKRate,MaturityDate,Principal,Cost,FairValue
1,Portfolio Company (1) (20),Business Description,Type of Investment (2) (3) (15),Investment Date (24),Shares/Units,Total Rate,Reference Rate and Spread (28),PIK Rate (19),Maturity Date,Principal (4),Cost (4),Fair Value (18)
2,Portfolio Company (1) (20),Business Description,Type of Investment (2) (3) (15),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Control Investments (5),Business Description,Type of Investment (2) (3) (15),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Control Investments (5),Business Description,Type of Investment (2) (3) (15),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Type of Investment (2) (3) (15),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Secured Debt,8/16/2019,NaN,15.38 %,10.00 % SF+,NaN,8/16/2024,$ 160,$ 159,$ 159
7,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Secured Debt,8/16/2019,NaN,15.38 %,10.00 % SF+,NaN,8/16/2024,4195,4137,4137
8,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Preferred Member Units,8/16/2019,3200,14.13 %,NaN,NaN,NaN,NaN,3200,—
9,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Preferred Member Units,5/20/2021,2427,NaN,NaN,NaN,NaN,NaN,2427,4580
10,"Analytical Systems Keco Holdings, LLC",Manufacturer of Liquid and Gas Analyzers,Warrants,8/16/2019,420,NaN,NaN,NaN,8/16/2029,NaN,316,—


## cik==1396440

In [74]:
import platform
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options

url = 'https://www.sec.gov/edgar/browse/?CIK=1396440'
options = Options()
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver.exe") \
    if platform.system() == "Linux" else webdriver.Chrome(options=options)
driver.get(url)
h5_tags = driver.find_elements(By.TAG_NAME,value='h5')

for h5_tag in h5_tags:
    if h5_tag.text == "[+] 10-K (annual reports) and 10-Q (quarterly reports)":
        # Click on the h5 tag.
        h5_tag.click()
        break
xpath = '//button[text()="View all 10-Ks and 10-Qs"]'
element = WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.XPATH,xpath)))
driver.execute_script("arguments[0].click();", element)

conditions = '@data-original-title="Open document" and contains(@href, "Archive") and not(contains(@href, "index")) and not(contains(@href, "xml"))'
table = driver.find_elements(By.CSS_SELECTOR,value='div.dataTables_scroll')

_from = driver.find_elements(By.ID,value='filingDateFrom')
_to = driver.find_elements(By.ID,value='filingDateTo')
_from[0].clear();_to[0].clear()


In [106]:
import os
import glob
import re
import warnings
import datetime
import pandas as pd 
import numpy as np

warnings.simplefilter(action='ignore', category=FutureWarning)

def stopping_criterion(
    criterias:tuple=('total investments','Invesmtents','total portfolio investments',)
)->str:
    tuple(map(lambda cr:cr.replace(' ',r'\s*'),criterias))
    return '{}|{}|{}'.format(*tuple(map(lambda cr:cr.replace(' ',r'\s*'),criterias)))


def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('senior secured loans',
        'first lien',
        'second lien',
        'senior secured bonds',
        'subordinated debt',
        'equity/other',
        'collateralized securities',
        'preferred equity—',
        'Equity/Warrants',
        'unsecured debt',
        'senior secured notes',
        'warrants',
        'total senior secured first lien term loans',
        'secured debt')
    ))

def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'control investments',
        'affiliate investments',
        'non-control/non-affilate investments',
        'Non-Controlled/Non-Affiliated  Investments:',
        'Affiliated  Investments:',
        'Non-Controlled/Non-Affiliated  Investments  :',
        'Affiliated  Investments  :',
        'Non-controlled/Non-affiliated Investments',
        'Affiliated Investments',
        )
    ))

# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'TypeofInvestment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None ,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    # df.drop(idx,axis=0,inplace=True,errors='ignore') 
    return df


def merge_duplicate_columns(
    df:pd.DataFrame,
)->pd.DataFrame:
    duplicate_cols = df.columns[df.columns.duplicated(keep=False)]
    for col_name in duplicate_cols.unique():
        duplicate_data = df.loc[:, df.columns == col_name]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, df.columns != col_name]
        df[col_name] = merged_data
    return df

def remove_row_duplicates(row):
    seen = set()
    return pd.Series([x if x not in seen and not seen.add(x) else None for x in row])

def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final


def _clean_qtr(
    file_path:str
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0)
    df.replace(['\u200b','',')',':','$','%',0],np.nan,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    df.dropna(axis=0,how='all',inplace=True)
    
    cols = df.iloc[0].str.replace('[^a-zA-Z]', '', regex=True)
    if ((cols == '') + (cols == 'nan') + (cols == np.nan) + (cols == 'NaN')).sum() > int(file_path.split('\\')[-3] == '1396440'):
        df = df.apply(remove_row_duplicates, axis=1)
    else:
        df.columns = cols
    df = merge_duplicate_columns(df)
    df.replace(['\u200b','',')',':','$','%',0],np.nan,inplace=True) #':','$','%'
    if ((cols == '') + (cols == 'nan') + (cols == np.nan) + (cols == 'NaN')).sum() > int(file_path.split('\\')[-3] == '1396440'):
        df.dropna(axis=1,how='all',inplace=True)
        df.dropna(axis=0,how='all',inplace=True)
    # df = df.iloc[1: , :]
    df.drop(columns=df.columns[pd.isna(df.columns)].tolist() + [col for col in df.columns if col == ''],axis=1,inplace=True)
    return df


cik = 1396440
ROOT_PATH = r'C:\Users\pysol\Desktop\projects\sec_filings'

def main()->None:
    bdc = os.path.join(ROOT_PATH,str(cik))
    qtrs = os.listdir(bdc)
    for qtr in qtrs:
        if '.csv' in qtr:
            continue
        # qtr = '2009-12-31'
        index_list_sum = 0
        i = 1
        df = _clean_qtr( os.path.join(bdc,qtr,f'Schedule_of_Investments_{i - 1}.csv'))
        col = df.columns.tolist()
        dfs = [df]
        while index_list_sum == 0:
            qtr_file = os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv')
            df = _clean_qtr(qtr_file)
            df.columns = col
            dfs.append(df)
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
        date_final = pd.concat(dfs,axis=0,ignore_index=True)#pd.DataFrame(concat(*dfs))   
        # Create a dictionary for renaming columns
        rename_dict = {col: 'portfoliocompany' for col in df.columns if 'portfolio' in col.lower()}
        # if str(cik) == '1396440':
        #     rename_dict.update({'BusinessDescription':'Industry'})
        # Rename the columns
        date_final.rename(columns=rename_dict, inplace=True)
        date_final = date_final[['portfoliocompany']+[col for col in date_final.columns if col != 'portfoliocompany' ]]
        date_final = extract_subheaders(date_final,control=True)
        date_final = extract_subheaders(date_final,control=False)
        date_final['qtr'] = qtr.split('\\')[-1]
        for i in range(3):
            date_final[date_final.columns[i]].fillna(method='ffill',inplace=True)
        if not os.path.exists(os.path.join(bdc,qtr,'output')):
            os.makedirs(os.path.join(bdc,qtr,'output'))
        date_final.to_csv(os.path.join(bdc,qtr,'output',f'{qtr}.csv'),index=False)
        # break
    # Use glob to find files
    files = sorted(glob.glob(f'{cik}/*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.to_csv(os.path.join(str(cik),f'{cik}_soi_table.csv'),index=False)
    
main()

C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\2434195389.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data
C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\2434195389.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data
C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\2434195389.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

## cik==1490349

In [51]:

import os
import glob
import warnings
import re
import datetime
import pandas as pd 
import numpy as np

def stopping_criterion(
    search_string:str='total investments'
)->str:
    # Regular expression to ignore whitespace and case
    regex_pattern = search_string.replace(' ', r'\s*')
    return '{}|{}'.format(regex_pattern,'Invesmtents')

def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('senior secured loans',
        'first lien',
        'second lien',
        'senior secured bonds',
        'subordinated debt',
        'equity/other',
        'collateralized securities',
        'preferred equity—',
        'Equity/Warrants',
        'unsecured debt',
        'senior secured notes',
        'warrants',
        'total senior secured first lien term loans')
    ))

def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'control investments',
        'affiliate investments',
        'non-control/non-affilate investments',
        'Non-Controlled/Non-Affiliated  Investments:',
        'Affiliated  Investments:',
        'Non-Controlled/Non-Affiliated  Investments  :',
        'Affiliated  Investments  :',
        'Non-controlled/Non-affiliated Investments',
        'Affiliated Investments',
        )
    ))

# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

def remove_row_duplicates(row):
    seen = set()
    return pd.Series([x if x not in seen and not seen.add(x) else None for x in row])

def merge_duplicate_columns(
    df:pd.DataFrame,
)->pd.DataFrame:
    duplicate_cols = df.columns[df.columns.duplicated(keep=False)]
    for col_name in duplicate_cols.unique():
        duplicate_data = df.loc[:, df.columns == col_name]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, df.columns != col_name]
        df[col_name] = merged_data
    return df

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'TypeofInvestment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    # df.drop(idx,axis=0,inplace=True,errors='ignore') 
    return df

def _clean_qtr(
    file_path:str
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0)
    df.replace(['\u200b','',')',':','$','%',0],np.nan,inplace=True) #':','$','%'
    df.dropna(axis=1,how='all',inplace=True)
    df.dropna(axis=0,how='all',inplace=True)
    
    cols = df.iloc[0].str.replace('[^a-zA-Z]', '', regex=True)
    if ((cols == '') + (cols == 'nan') + (cols == np.nan) + (cols == 'NaN')).sum() > int(file_path.split('\\')[-3] == '1396440'):
        df = df.apply(remove_row_duplicates, axis=1)
    else:
        df.columns = cols
    df = merge_duplicate_columns(df)
    df.replace(['\u200b','',')',':','$','%',0],np.nan,inplace=True) #':','$','%'
    if ((cols == '') + (cols == 'nan') + (cols == np.nan) + (cols == 'NaN')).sum() > int(file_path.split('\\')[-3] == '1396440'):
        df.dropna(axis=1,how='all',inplace=True)
        df.dropna(axis=0,how='all',inplace=True)
    df.drop(columns=df.columns[pd.isna(df.columns)].tolist() + [col for col in df.columns if col == ''],axis=1,inplace=True)
    return df

cik = 1490349
ROOT_PATH = r'C:\Users\pysol\Desktop\projects\sec_filings'

def main()->None:
    bdc = os.path.join(ROOT_PATH,str(cik))
    qtrs = os.listdir(bdc)
    for qtr in qtrs:
        if '.csv' in qtr or not os.path.exists(os.path.join(bdc,qtr,f'Schedule_of_Investments_0.csv')):
            continue
        # qtr = '2013-06-30'
        dfs = []
        index_list_sum = i = 0
        col = None
        while index_list_sum == 0:
            qtr_file = os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv')
            df = _clean_qtr(qtr_file)
            if i == 0:
                col = df.columns.tolist()
            else:
                df.columns = col
            dfs.append(df)
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
        
        date_final = pd.concat(dfs,axis=0,ignore_index=True)#pd.DataFrame(concat(*dfs))
        date_final = extract_subheaders(date_final,control=True)
        date_final = extract_subheaders(date_final,control=False)

        date_final['qtr'] = qtr.split('\\')[-1]
        for i in range(3):
            date_final[date_final.columns[i]].fillna(method='ffill',inplace=True)
        if not os.path.exists(os.path.join(bdc,qtr,'output')):
            os.makedirs(os.path.join(bdc,qtr,'output'))
        date_final.to_csv(os.path.join(bdc,qtr,'output',f'{qtr}.csv'),index=False)
        # break
    
    # Use glob to find files
    files = sorted(glob.glob(f'{cik}/*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.to_csv(os.path.join(str(cik),f'{cik}_soi_table.csv'),index=False)
    
main()


C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\324653766.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data
C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\324653766.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = merged_data
C:\Users\pysol\AppData\Local\Temp\ipykernel_27936\324653766.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

## cik==1379785

In [28]:
import os
import re
import glob
import datetime
import pandas as pd
import numpy as np


def stopping_criterion(
    search_string:str='total investments'
)->str:
    # Regular expression to ignore whitespace and case
    regex_pattern = search_string.replace(' ', r'\s*')
    return '{}|{}'.format(regex_pattern,'Invesmtents')

def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('senior secured loans',
        'first lien',
        'second lien',
        'senior secured bonds',
        'subordinated debt',
        'equity/other',
        'collateralized securities',
        'preferred equity—',
        'Equity/Warrants',
        'unsecured debt',
        'senior secured notes',
        'warrants',
        'total senior secured first lien term loans')
    ))

def standard_field_names()->tuple:
    return (
        'portfolio',
        'footnotes',
        'industry',
        'rate',
        'floor',
        'maturity',
        'principal amount', # TODO change stand names for more dynamic fuzzywuzzy matching
        'cost',
        'value',
        'investment',
        'date',
        'subheaders',
        'number of shares'
    )

def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'control investments',
        'affiliate investments',
        'non-control/non-affilate investments',
        'Non-Controlled/Non-Affiliated  Investments:',
        'Affiliated  Investments:',
        'Non-Controlled/Non-Affiliated  Investments  :',
        'Affiliated  Investments  :',
        'Non-controlled/Non-affiliated Investments',
        'Affiliated Investments',
        )
    ))

# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

def remove_row_duplicates(row):
    seen = set()
    return pd.Series([x if x not in seen and not seen.add(x) else None for x in row])

def shift_part(row,left,after=3):
    # Keeping the first three values unchanged
    first_part = row[:after]
    # Shifting the remaining part of the row to the left by 1
    shifted_part = row[after:].shift(-1 if left else 1)
    # Concatenating the two parts
    return pd.concat([first_part, shifted_part])



def merge_duplicate_columns(
    df:pd.DataFrame,
)->pd.DataFrame:
    duplicate_cols = df.columns[df.columns.duplicated(keep=False)]
    for col_name in duplicate_cols.unique():
        duplicate_data = df.loc[:, df.columns == col_name]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, df.columns != col_name]
        df[col_name] = merged_data
    return df

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'TypeofInvestment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    # df.drop(idx,axis=0,inplace=True,errors='ignore') 
    return df


def clean_bbdc(
    file_path:str
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0)
    df.dropna(axis=1,how='all',inplace=True)
    df.dropna(axis=0,how='all',inplace=True)
    df.replace(['\u200b','%',],np.nan,inplace=True) #':','$','%'

    if not '2012-06-30' in file_path:
        df[df.apply(lambda row: row.astype(str).str.contains('TRIANGLE CAPITAL CORPORATION').any(), axis=1)] = ''
        regex_pattern = '|'.join(map(re.escape, standard_field_names()))
        mask = df.iloc[:,0].str.contains(regex_pattern, case=False, na=False)
        df.iloc[0] = df.iloc[:mask.idxmax()].astype(str).agg(' '.join)
        df.reset_index(drop=True,inplace=True)
        df.drop(axis=0, index=1,inplace=True)

    for i in range(df.shape[0]):
        df.iloc[i] = [np.nan if  item in ['None None','nan nan'] else str(item).replace('nan','').replace('None','') for item in df.iloc[i]]
        
    if '2010-09-30/Schedule_of_Investments_0.csv' in file_path:
        df.iloc[0] = shift_part(df.iloc[0],left=True)
    if '2010-09-30/Schedule_of_Investments_1.csv' in file_path:
        df.iloc[0] = shift_part(df.iloc[0],left=False,after=8)

        
    df.columns = df.iloc[0].astype(str)
    df = merge_duplicate_columns(df)
    df = df.apply(remove_row_duplicates, axis=1)
    non_empty_counts = df.applymap(lambda x: x != '' and pd.notna(x)).sum()
    return df.loc[:, non_empty_counts > 3]

cik = 1379785
ROOT_PATH = r'C:\Users\pysol\Desktop\projects\sec_filings'

def main()->None:
    bdc = os.path.join(ROOT_PATH,str(cik))
    qtrs = os.listdir(bdc)
    for qtr in qtrs:
        if '.csv' in qtr or not os.path.exists(os.path.join(bdc,qtr,f'Schedule_of_Investments_0.csv')):
            continue
        dfs = []
        index_list_sum = i = 0
        qtr = '2010-09-30'
        while index_list_sum == 0:
            qtr_file = os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv')
            df = clean_bbdc(qtr_file)
            dfs.append(df)
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
        
        date_final = pd.DataFrame(concat(*dfs))
        date_final = extract_subheaders(date_final,control=True)

        date_final['qtr'] = qtr.split('\\')[-1]
        for i in range(3):
            date_final[date_final.columns[i]].fillna(method='ffill',inplace=True)
        if not os.path.exists(os.path.join(bdc,qtr,'output')):
            os.makedirs(os.path.join(bdc,qtr,'output'))
        date_final.to_csv(os.path.join(bdc,qtr,'output',f'{qtr}.csv'),index=False)
        break
    # Use glob to find files
    files = sorted(glob.glob(f'{cik}/*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.to_csv(os.path.join(str(cik),f'{cik}_soi_table.csv'),index=False)
    
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
# main()

In [ ]:
        df.columns = df.iloc[0].astype(str)
    if 'nan' in df.columns:
        df.drop('nan',axis=1,inplace=True)
    if not '2012-06-30' in file_path:
        df[df.apply(lambda row: row.astype(str).str.contains('TRIANGLE CAPITAL CORPORATION').any(), axis=1)] = np.nan
        regex_pattern = '|'.join(map(re.escape, standard_field_names()))
        mask = df.iloc[:,0].str.contains(regex_pattern, case=False, na=False)
        df.iloc[0] = df.iloc[:mask.idxmax()].astype(str).agg(' '.join)
        df.reset_index(drop=True,inplace=True)
        df.drop(axis=0, index=1,inplace=True)

In [101]:
import re
import pandas as pd
import numpy as np
from lxml import html


html_file = r'C:\Users\pysol\Desktop\projects\sec_filings\1379785\2010-09-30\g25057e10vq.html'
xpaths_file = r'C:\Users\pysol\Desktop\projects\sec_filings\xpaths\1379785.txt'

def read_elements(
    html_file:str,
    xpaths_file:str,
)->list:
    with open(html_file,'r',encoding='utf-8') as file:
        doc = file.read()

    with open(xpaths_file,'r',encoding='utf-8') as file:
        xpaths = file.readlines()

    tree = html.fromstring(doc)

    elements = []
    for xpath in xpaths:
        elements.extend(tree.xpath(xpath))
    return elements

elements = read_elements(html_file,xpaths_file)
type(elements[0])

lxml.html.HtmlElement

In [107]:
'''
TODO 
Try column based tag extraction
    identify column it belongs to based on value type 
    know the order of column to be
    
look in to args of pd.read_html
'''
def element_extract(
    element:html.HtmlElement
)->pd.DataFrame:
    data = []
    for row in element.findall(".//tr"):
        cells = row.findall(".//td")
        row_data = [] 
        for cell in cells:
            content = re.sub(r"\s+", "", cell.text_content())
            content = re.sub(r"\\n+", "", content)
            content = re.sub(',', '',content)
            row_data.append(content)
        data.append(row_data)
    return pd.DataFrame(data)

def prelim_clean(
    df:pd.DataFrame,
)->pd.DataFrame:
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.replace('', np.nan,inplace=True)
    df.replace({None: np.nan},inplace=True)
    df.dropna(axis=1,how='all',inplace=True)
    df.dropna(axis=0,how='all',inplace=True)
    return df.reset_index(drop=True)


df = prelim_clean(element_extract(elements[2]))
df

C:\Users\pysol\AppData\Local\Temp\ipykernel_25236\1410006369.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\pysol\AppData\Local\Temp\ipykernel_25236\1410006369.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('', np.nan,inplace=True)


,0,2,4,6,7,9,10,11,12,13,14,15,17,19
0,PortfolioCompany,Industry,TypeofInvestment(1)(2),PrincipalAmount,NaN,Cost,NaN,NaN,FairValue(3),NaN,NaN,NaN,NaN,NaN
1,MediaTempleInc.(7%)*,WebHostingServices,SubordinatedNote(12%Cash4%PIKDue04/15),$,8800000,NaN,$,8616959,NaN,NaN,$,8616959,NaN,NaN
2,NaN,NaN,NaN,ConvertibleNote(8%Cash4%PIKDue04/15),NaN,3200000,NaN,NaN,NaN,2642547,NaN,NaN,2642547,NaN
3,NaN,NaN,NaN,CommonStockPurchaseWarrant(28000Shares),NaN,NaN,NaN,NaN,NaN,536000,NaN,NaN,536000,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12000000,NaN,NaN,NaN,11795506,NaN,11795506
5,MincoTechnologyLabsLLC(3%)*,SemiconductorDistribution,SubordinatedNote(13%Cash3.25%PIKDue05/16),NaN,5060188,NaN,NaN,4938707,NaN,NaN,NaN,4938707,NaN,NaN
6,NaN,NaN,NaN,ClassAUnits(5000Units),NaN,NaN,NaN,NaN,NaN,500000,NaN,NaN,500000,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5060188,NaN,NaN,NaN,5438707,NaN,5438707
8,NovolyteTechnologiesInc.(5%)*,SpecialtyManufacturing,SubordinatedNote(12%Cash5.5%PIKDue04/15),NaN,7677817,NaN,NaN,7569278,NaN,NaN,NaN,7569278,NaN,NaN
9,NaN,NaN,NaN,PreferredUnits(641units),NaN,NaN,NaN,NaN,NaN,640818,NaN,NaN,640818,NaN


In [106]:

# string_fields = (        
#     'portfolio_company',
#     # 'industry',
#     'type_of_investment',
# )

def divide_evenly(number, parts):
    if parts <= 0:
        raise ValueError("Number of parts must be greater than 0.")
    if number < 0:
        raise ValueError("Number must be non-negative.")
        
    base_part = number // parts
    extra_needed = number % parts
    
    # Parts that get an extra unit to handle the remainder
    parts_with_extra = [base_part + 1] * extra_needed
    # Parts that don't get the extra unit
    parts_without_extra = [base_part] * (parts - extra_needed)
    
    # Concatenate the two parts
    result = parts_with_extra + parts_without_extra
    
    return result


# Function to attempt conversion and determine type
def tag_column_name(
    df:pd.DataFrame,
    numeric_fields:tuple=('principal_amount','cost','fair_value',)
)->str:
    
    numeric_cols = [
        df.loc[1:,col].str.isnumeric().any() or df.loc[1:,col].isna().all()
        for col in df.columns
    ]
    # Separating the DataFrame into two based on the type of columns
    df_numeric,df_non_numeric = df.loc[:,numeric_cols],df.loc[:,[not value for value in numeric_cols]]
    # numeric_fields = df_numeric.iloc[0][df_numeric.iloc[0].notna()].tolist()
    string_fields = df_non_numeric.iloc[0][df_non_numeric.iloc[0].notna()].tolist()
    string_fields.remove('Industry')
    # num_div = divide_evenly(df_numeric.shape[1],  len(numeric_fields))
    num_div = round(df_numeric.shape[1] /  len(numeric_fields))
    print(numeric_fields,string_fields,num_div,df_numeric.shape[1],  len(numeric_fields))
        
    # numeric_mask = df_numeric.applymap(lambda x: x != '' and pd.notna(x)).sum()
    # string_mask = df_non_numeric.applymap(lambda x: x != '' and pd.notna(x)).sum()
    # df_numeric,df_non_numeric = df_numeric.loc[:,numeric_mask > 1],df_non_numeric.loc[:,string_mask > 1]
    
    i,num_cols = 0,[None]*df_numeric.shape[1]
    start,end = i*num_div,min((i + 1)*num_div,df_numeric.shape[1])
    while i*len(numeric_fields) - df_numeric.shape[1] and start < end:
        num_cols[start:end] = [numeric_fields[i]]*(end - start)
        i += 1
        start,end = i*num_div,min((i + 1)*num_div,df_numeric.shape[1])
    df_numeric.columns = num_cols
    
    flag,itr,columns = False,0,[None]*len(df_non_numeric.columns)
    for i,col in enumerate(df_non_numeric.columns):
        flag = df_non_numeric.loc[:,col].str.contains('%/*()',regex=True).any()
        columns[i] = string_fields[itr]
        if not flag:
            columns[i] = 'industry'
            itr = int(not flag)
    df_non_numeric.columns = columns
                
    return pd.concat([df_non_numeric,df_numeric],axis=1)
        


tag_column_name(df)


('principal_amount', 'cost', 'fair_value') ['PortfolioCompany', 'TypeofInvestment(1)(2)', 'PrincipalAmount'] 3 8 3


C:\Users\pysol\AppData\Local\Temp\ipykernel_25236\864796125.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  flag = df_non_numeric.loc[:,col].str.contains('%/*()',regex=True).any()


,PortfolioCompany,industry,TypeofInvestment(1)(2),TypeofInvestment(1)(2),industry,industry,principal_amount,principal_amount,principal_amount,cost,cost,cost,fair_value,fair_value
0,PortfolioCompany,Industry,TypeofInvestment(1)(2),PrincipalAmount,NaN,NaN,NaN,Cost,NaN,FairValue(3),NaN,NaN,NaN,NaN
1,MediaTempleInc.(7%)*,WebHostingServices,SubordinatedNote(12%Cash4%PIKDue04/15),$,$,$,8800000,NaN,8616959,NaN,NaN,8616959,NaN,NaN
2,NaN,NaN,NaN,ConvertibleNote(8%Cash4%PIKDue04/15),NaN,NaN,NaN,3200000,NaN,NaN,2642547,NaN,2642547,NaN
3,NaN,NaN,NaN,CommonStockPurchaseWarrant(28000Shares),NaN,NaN,NaN,NaN,NaN,NaN,536000,NaN,536000,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12000000,NaN,NaN,11795506,NaN,11795506
5,MincoTechnologyLabsLLC(3%)*,SemiconductorDistribution,SubordinatedNote(13%Cash3.25%PIKDue05/16),NaN,NaN,NaN,5060188,NaN,4938707,NaN,NaN,4938707,NaN,NaN
6,NaN,NaN,NaN,ClassAUnits(5000Units),NaN,NaN,NaN,NaN,NaN,NaN,500000,NaN,500000,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5060188,NaN,NaN,5438707,NaN,5438707
8,NovolyteTechnologiesInc.(5%)*,SpecialtyManufacturing,SubordinatedNote(12%Cash5.5%PIKDue04/15),NaN,NaN,NaN,7677817,NaN,7569278,NaN,NaN,7569278,NaN,NaN
9,NaN,NaN,NaN,PreferredUnits(641units),NaN,NaN,NaN,NaN,NaN,NaN,640818,NaN,640818,NaN


In [100]:

def divide_evenly(number, parts):
    if parts <= 0:
        raise ValueError("Number of parts must be greater than 0.")
    if number < 0:
        raise ValueError("Number must be non-negative.")
        
    base_part = number // parts
    extra_needed = number % parts
    
    # Parts that get an extra unit to handle the remainder
    parts_with_extra = [base_part + 1] * extra_needed
    # Parts that don't get the extra unit
    parts_without_extra = [base_part] * (parts - extra_needed)
    
    # Concatenate the two parts
    result = parts_with_extra + parts_without_extra
    
    return result

divide_evenly(8,3)

[3, 3, 2]

## SEC API

In [1]:
from sec_api import ExtractorApi

api_key = 'ad73111c598021aa3c07419e067f727ae15d4427e8b6645af4ff83a83213d5b5'
extractorApi = ExtractorApi(api_key)

test_qtr = 'https://www.sec.gov/Archives/edgar/data/0001379785/000119312512101223/d310549d10k.htm'
help(extractorApi)

Help on ExtractorApi in module sec_api.index object:

class ExtractorApi(builtins.object)
 |  ExtractorApi(api_key, proxies=None)
 |  
 |  Base class for 10-K/10-Q/8-K item/section extractor API
 |  
 |  Methods defined here:
 |  
 |  __init__(self, api_key, proxies=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_section(self, filing_url='', section='1A', return_type='text')
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [7]:
test = extractorApi.get_section(
    filing_url=test_qtr, 
    section="1", 
    return_type="text",
)
print(test)

 Item 1. Business. 

We are a Maryland corporation incorporated on October 10, 2006, for the purposes of acquiring 100% of the equity interests in Triangle Mezzanine Fund LLLP (&#147;Triangle SBIC&#148;) and its general partner, Triangle Mezzanine LLC (&#147;TML&#148;), raising capital in our initial public offering, which was completed in February 2007 (the &#147;IPO&#148;) and thereafter operating as an internally managed business development company (&#147;BDC&#148;) under the Investment Company Act of 1940 (the &#147;1940 Act&#148;). On December 15, 2009, Triangle Mezzanine Fund II, LP (&#147;Triangle SBIC II&#148;) was organized as a limited partnership under the laws of the State of Delaware. Triangle SBIC&#146;s Small Business Investment Company (&#147;SBIC&#148;) license from the United States Small Business Administration (the &#147;SBA&#148;) became effective on September 11, 2003 and Triangle SBIC II&#146;s SBIC license became effective on May 26, 2010. Unless otherwise note

In [15]:
from sec_api import FullTextSearchApi

fullTextSearchApi = FullTextSearchApi(api_key=api_key)
query = {
  "query": '"LPCN 1154"',
  "formTypes": ['8-K', '10-Q'],
  "startDate": '2021-01-01',
  "endDate": '2021-06-14',
}
filings = fullTextSearchApi.get_filings(query)
filings

{'total': {'value': 3, 'relation': 'eq'},
 'filings': [{'accessionNo': '0001104659-21-080527',
   'cik': '1535955',
   'companyNameLong': 'Lipocine Inc. (LPCN) (CIK 0001535955)',
   'ticker': 'LPCN',
   'description': 'EXHIBIT 99.1',
   'formType': '8-K',
   'type': 'EX-99.1',
   'filingUrl': 'https://www.sec.gov/Archives/edgar/data/1535955/000110465921080527/tm2119438d1_ex99-1.htm',
   'filedAt': '2021-06-14'},
  {'accessionNo': '0001104659-21-080525',
   'cik': '1535955',
   'companyNameLong': 'Lipocine Inc. (LPCN) (CIK 0001535955)',
   'ticker': 'LPCN',
   'description': 'EXHIBIT 99.1',
   'formType': '8-K',
   'type': 'EX-99.1',
   'filingUrl': 'https://www.sec.gov/Archives/edgar/data/1535955/000110465921080525/tm2119156d1_ex99-1.htm',
   'filedAt': '2021-06-14'},
  {'accessionNo': '0001104659-21-080527',
   'cik': '1535955',
   'companyNameLong': 'Lipocine Inc. (LPCN) (CIK 0001535955)',
   'ticker': 'LPCN',
   'description': 'FORM 8-K',
   'formType': '8-K',
   'type': '8-K',
   '

## cik == 1490927

In [44]:
import os
import re
import glob
import datetime
import pandas as pd
import numpy as np
from fuzzywuzzy import process

def get_standard_name(col, choices, score_cutoff=60):
    best_match, score = process.extractOne(col, choices)
    if score > score_cutoff:
        return best_match
    return col

def stopping_criterion(
    search_string:str='total investments'
)->str:
    # Regular expression to ignore whitespace and case
    regex_pattern = search_string.replace(' ', r'\s*')
    return '{}|{}|{}|{}'.format(regex_pattern,'Invesmtents',r'130.4%\s*(b)',r'178.1%\s*(b)',r'Total\s*Control\s*&\s*Affiliate\s*Investments')

def concat(*dfs)->list:
    final = []
    for df in dfs:
        final.extend(df.values.tolist())
    return final

def common_subheaders()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        ('senior secured loans',
        'first lien',
        'second lien',
        'senior secured bonds',
        'subordinated debt',
        'equity/other',
        'collateralized securities',
        'preferred equity—',
        'Equity/Warrants',
        'unsecured debt',
        'senior secured notes',
        'warrants',
        'total senior secured first lien term loans')
    ))

def standard_field_names()->tuple:
    return (
        'portfolio',
        'footnotes',
        'industry',
        'rate',
        'floor',
        'maturity',
        'principal amount', # TODO change stand names for more dynamic fuzzywuzzy matching
        'cost',
        'value',
        'investment',
        'date',
        'subheaders',
        'number of shares'
    )

def company_control_headers()->tuple:
    return tuple(map(lambda header:header.replace(' ', r'\s*'),
        (
        'control investments',
        'affiliate investments',
        'non-control/non-affilate investments',
        'Non-Controlled/Non-Affiliated  Investments:',
        'Affiliated  Investments:',
        'Non-Controlled/Non-Affiliated  Investments  :',
        'Affiliated  Investments  :',
        'Non-controlled/Non-affiliated Investments',
        'Affiliated Investments',
        )
    ))

def strip_string(
    columns_names:list,
    standardize:bool=False
)->tuple:
    columns = tuple(map(lambda col:re.sub(r'[^a-z]', '', str(col).lower()),columns_names))
    if standardize:
        standard_fields = standard_field_names()
        return tuple(
            get_standard_name(col,standard_fields) for col in columns
        )
    return columns

def get_key_fields(
    df_cur:pd.DataFrame
)->tuple:
    important_fields = standard_field_names() + common_subheaders()
    for idx,row in enumerate(df_cur.iterrows()):
        found = any(any(
            key in str(field).lower() 
            for key in important_fields)
                    for field in row[-1].dropna().tolist()
            )
        if found and len(set(row[-1].dropna().tolist())) >= 5:
            fields = strip_string(row[-1].tolist(),standardize=found) ,idx
            return fields
    return strip_string(df_cur.iloc[0].tolist(),standardize=found),0

# Function to extract date and convert to datetime object
def extract_date(file_path):
    # Extract date from file path (assuming date is always in 'YYYY-MM-DD' format)
    date_str = re.search(r'\d{4}-\d{2}-\d{2}', file_path).group()
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

def remove_row_duplicates(row):
    seen = set()
    return pd.Series([x if x not in seen and not seen.add(x) else None for x in row])


def merge_duplicate_columns(
    df:pd.DataFrame,
)->pd.DataFrame:
    duplicate_cols = df.columns[df.columns.duplicated(keep=False)]
    for col_name in duplicate_cols.unique():
        duplicate_data = df.loc[:, df.columns == col_name]
        merged_data = duplicate_data.apply(lambda row: ' '.join(set(row.dropna().astype(str))), axis=1)
        df = df.loc[:, df.columns != col_name]
        df[col_name] = merged_data
    return df

def extract_subheaders(
    df:pd.DataFrame,
    control:bool,
)->pd.DataFrame:
    col_name = 'company_control' if control else 'TypeofInvestment'
    if col_name in df.columns:
        return df
    include = df.apply(
        lambda row: re.search('|'.join(company_control_headers() if control else common_subheaders()), str(row[0]), re.IGNORECASE) is not None,
        axis=1
    )  
    
    exclude = ~df.apply(
        lambda row: row.astype(str).str.contains('total|Inc|Ltd|LLC|Holdings|LP|Co|Corporation', case=False, na=False).any(),
        axis=1
    )
    idx = df[include & exclude].index.tolist()
    df[col_name] = None
    if not idx:
        return df

    prev_header = subheader = None
    df.loc[idx[-1]:,col_name] = df.iloc[idx[-1],1] if isinstance(df.iloc[idx[-1],0],float)  else df.iloc[idx[-1],0]
    for j,i in enumerate(idx[:-1]):
        prev_header = subheader
        subheader = df.iloc[i,1] if isinstance(df.iloc[i,0],float)  else df.iloc[i,0]
        df.loc[idx[j]:idx[j+1],col_name] = subheader if subheader != '' else prev_header
    # df.drop(idx,axis=0,inplace=True,errors='ignore') 
    return df

def _clean(
    file_path:str,
    regex_pattern:str=r'\w+\s+\d{1,2},\s+\d{4}'
)->pd.DataFrame:
    df = pd.read_csv(file_path,index_col=0)
    df.dropna(axis=0,how='all',inplace=True)
    df = df[~df.apply(lambda row:row.str.match(regex_pattern).all(),axis=1)]

    if '2013-06-30\Schedule_of_Investments_1.csv' in file_path:
        df = df.iloc[:,[0,2,4,6,10,14,18]]
        df.columns = list(range(df.shape[1]))
        return df

    important_fields,idx = get_key_fields(df)
    df.columns = important_fields
    # if '2018-06-30' in file_path:
    #     df.replace(['\u200b','%',' ',np.nan,'\n','\\'],'',inplace=True)
    #     df.columns = df.iloc[:5,:].apply(lambda x: ''.join(x), axis=0)
    
    df = merge_duplicate_columns(df)
    df.columns = list(range(df.shape[1]))
    df.replace(['\u200b','%',' ',''],np.nan,inplace=True) #':','$','%'
    columns = (~df.isna()).sum(axis=0) <= 1
    df.drop(columns=columns[columns].index,inplace=True)
    return df.reset_index(drop=True)


cik = 1490927
ROOT_PATH = r'C:\Users\pysol\Desktop\projects\sec_filings'

def main()->None:
    bdc = os.path.join(ROOT_PATH,str(cik))
    qtrs = os.listdir(bdc)
    for qtr in qtrs:
        if '.csv' in qtr or not os.path.exists(os.path.join(bdc,qtr,f'Schedule_of_Investments_0.csv')):
            continue
        index_list_sum = i = 0
        df = _clean(os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv'))
        dfs = [df]     
        i += 1
        while index_list_sum == 0 and os.path.exists(os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv')):
            dfs.append(df)
            df = _clean(os.path.join(bdc,qtr,f'Schedule_of_Investments_{i}.csv'))
            index_list = df.apply(
                lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
                axis=1
            )
            index_list_sum = index_list.sum()
            i += 1
            
        date_final = dfs[0]
        if len(dfs) > 1:
            date_final = pd.concat(dfs,axis=0,ignore_index=True)#pd.DataFrame(concat(*dfs))
        date_final = extract_subheaders(date_final,control=True)
        date_final = extract_subheaders(date_final,control=False)

        date_final['qtr'] = qtr.split('\\')[-1]
        for i in range(3):
            date_final[date_final.columns[i]].fillna(method='ffill',inplace=True)
        if not os.path.exists(os.path.join(bdc,qtr,'output')):
            os.makedirs(os.path.join(bdc,qtr,'output'))
        date_final.replace([None,'\u200b',''],np.nan,inplace=True) #':','$','%'
        columns_to_drop = date_final.notna().sum() <= 2
        date_final.drop(columns=columns_to_drop[columns_to_drop].index)
        date_final.to_csv(os.path.join(bdc,qtr,'output',f'{qtr}.csv'),index=False)
    # Use glob to find files
    files = sorted(glob.glob(f'{cik}/*/output/*.csv'), key=extract_date)
    single_truth = pd.concat([
        pd.read_csv(df) for df in files
    ],axis=0,ignore_index=True)
    single_truth.to_csv(os.path.join(str(cik),f'{cik}_soi_table.csv'),index=False)
    
    
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
main()

C:\Users\pysol\AppData\Local\Temp\ipykernel_4960\108091981.py:201: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
C:\Users\pysol\AppData\Local\Temp\ipykernel_4960\108091981.py:201: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
C:\Users\pysol\AppData\Local\Temp\ipykernel_4960\108091981.py:201: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row:row.astype(str).str.contains(stopping_criterion(), case=False, na=False).any(),
C:\Users\pysol\AppData\Local\Temp\ipykernel_4960\108091981.py:201: UserWarning: This pattern is interpreted as a regular 